In [100]:
import pyterrier as pt
import pandas as pd
import numpy as np

import lightgbm as lgb
from sklearn.model_selection import train_test_split

from copy import deepcopy

In [3]:
if not pt.started():
    pt.init()

PyTerrier 0.4.0 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


In [13]:
ds = pt.get_dataset("trec-deep-learning-passages")
index = pt.IndexFactory.of("./passage_index_8_bkp/data.properties")

14:30:50.665 [main] WARN  o.t.structures.CompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 2.1 GiB of memory would be required.


In [14]:
print(index.getCollectionStatistics().toString())

Number of documents: 8841823
Number of terms: 1170682
Number of postings: 215238456
Number of fields: 1
Number of tokens: 288759529
Field names: [text]
Positions:   false



In [70]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

In [16]:
pt.Experiment(
    [bm25],
    ds.get_topics('test-2019'),
    ds.get_qrels('test-2019'),
    eval_metrics=["map", "recip_rank", "ndcg"],
    names = ['BM25']
)

,name,map,recip_rank,ndcg
0,BM25,0.369987,0.795028,0.59338


In [62]:
df = ds.get_topics('train')
df.head()

,qid,query
0,121352,define extreme
1,634306,what does chattel mean on credit history
2,920825,what was the great leap forward brainly
3,510633,tattoo fixers how much does it cost
4,737889,what is decentralization process


In [63]:
df.dtypes

qid      object
query    object
dtype: object

In [69]:
df['qid'].unique()

array(['121352', '634306', '920825', ..., '210839', '908165', '50393'],
      dtype=object)

In [48]:
qrels = ds.get_qrels('train')
qrels.head()

,qid,docno,label
0,1185869,0,1
1,1185868,16,1
2,597651,49,1
3,403613,60,1
4,1183785,389,1


In [90]:
df = df.astype({'qid': 'int64'})

In [56]:
qrels.dtypes

qid      object
docno    object
label     int64
dtype: object

In [58]:
full_table = df.join(qrels, on='qid', how='right', lsuffix='_topics', rsuffix='_qrels')

In [59]:
full_table.head()

,qid,qid_topics,query,qid_qrels,docno,label
NaN,0,NaN,NaN,1185869,0,1
322300.0,1,1.0,a potlatch is considered an example of,1185868,16,1
NaN,2,NaN,NaN,597651,49,1
36118.0,3,3.0,another name for the primary visual cortex is,403613,60,1
455738.0,4,4.0,defining alcoholism as a disease is associated...,1183785,389,1


In [88]:
test_queries = ds.get_topics('test-2019')
test_queries = test_queries.astype({'qid': 'int64'})
test_queries.head()

,qid,query
0,1108939,what slows down the flow of blood
1,1112389,what is the county for grand rapids mn
2,792752,what is ruclip
3,1119729,what do you do when you have a nosebleed from ...
4,1105095,where is sugar lake lodge located


In [121]:
test_qrels = ds.get_qrels('test-2019')
test_qrels = test_qrels.astype({'qid': 'int64'})

In [138]:
Q = test_queries.iloc[55]

In [139]:
Q.qid

1133167

In [140]:
bm25_t = pt.BatchRetrieve(index, wmodel="BM25") % 4
bm25_t = bm25_t.compile()

Applying 8 rules


In [142]:
bm25_t.search(Q.query)

,qid,docid,docno,rank,score,query
0,1,5356601,5674622,0,33.297181,how is the weather in jamaica
1,1,2823018,8160230,1,31.210468,how is the weather in jamaica
2,1,1369965,8255705,2,31.180305,how is the weather in jamaica
3,1,1498595,190809,3,30.632402,how is the weather in jamaica


In [143]:
Q_qrels = test_qrels[test_qrels.qid == Q.qid]
Q_qrels.head()

,qid,docno,label
8768,1133167,1014506,0
8769,1133167,1060131,0
8770,1133167,1064355,0
8771,1133167,1070102,0
8772,1133167,1097959,0


In [146]:
Q_qrels = Q_qrels.astype({'docno': 'int64'})

In [149]:
Q_qrels[Q_qrels.docno == 5674622]

,qid,docno,label
9040,1133167,5674622,2
